In [1]:
import xarray as xr
import plotly.express as px
import datashader as dsh
import numpy as np
import pandas as pd
import colorcet
ds = xr.open_dataset('cnemc.h5')

## 转为网格均值数据

In [ ]:
def df_to_grid(df, var='pm2_5'):
    left = 70  # 左边界
    right = 140  # 右边界
    top = 55  # 上边界
    bottom = 15  # 下边界
    width = 280  # 东西向网格数
    height = 160  # 南北向网格数

    # 取边界范围内的数据
    dff = df.query(f'latitude < {top}')\
            .query(f'latitude > {bottom}')\
            .query(f'longitude > {left}')\
            .query(f'longitude < {right}')

    # 根据边界范围和网格数，准备栅格化后的画布
    cvs = dsh.Canvas(plot_width=width,
                     plot_height=height,
                     x_range=[left, right],
                     y_range=[bottom, top],
                     x_axis_type='linear',
                     y_axis_type='linear')

    # 将站点值放置到画布上
    agg = cvs.points(dff,
                     x='longitude',
                     y='latitude',
                     agg=dsh.mean(var))
    if 'timepoint' in df.index:
        agg['time'] = df.index.get_level_values('timepoint').unique()[0]
    else:
        agg['time'] = df['timepoint'].unique()[0]
    agg['name'] = var
    return agg

In [ ]:
aggs = []
for i in range(len(ds.timepoint)):
    ds_tmp = ds.isel(timepoint=i)
    df = ds_tmp.to_dataframe()
    agg = df_to_grid(df)
    aggs.append(agg)

ds_grid = xr.concat(aggs, 'time')

In [ ]:
ds_grid.mean('time').plot()

In [ ]:
fig = px.imshow(ds_grid[-100:], animation_frame='time',
                width=800, height=600,
                zmin=0, zmax=100,
                aspect='equal',
                origin='lower',
                color_continuous_scale='viridis')
fig.show()

## 对某个区域的站点数据进行时序分析

In [ ]:
# 直接使用 xarray 数据
ds_tmp = ds['pm2_5'][ds['area'] == '北京市']

In [ ]:
fig = px.imshow(ds_tmp, color_continuous_scale='RdBu_r',
                zmin=0, zmax=200,
                aspect='equal')
fig.show()

In [ ]:
# 转为表格可绘制更多类型
df_tmp = ds['no2'].to_dataframe().reset_index()
df_tmp = df_tmp[df_tmp.area.isin(['北京市', '上海市', '广州市', '南通市', '徐州市'])]

In [ ]:
fig = px.line(df_tmp, x="timepoint", y="no2",
              color="area",
              line_group="positionname",
              hover_name="positionname",
              line_shape='spline',
              # markers=True,
              render_mode="svg")
fig.show()

## 将站点数据表示在地图上

In [ ]:
ds_tmp = ds['pm2_5'].mean(dim='timepoint') - ds['pm2_5'].mean()  # 站点均值减去所有数据均值
df_tmp = ds_tmp.to_dataframe().reset_index()

In [ ]:
fig = px.scatter_mapbox(df_tmp,
                        lat="latitude",
                        lon="longitude",
                        hover_name="positionname",
                        color="pm2_5",
                        range_color=(-100, 100),
                        mapbox_style="open-street-map",
                        width=1200, height=600,
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=35, zoom=5)
fig.show()

In [ ]:
# 也可以直接导出数据到 https://kepler.gl/demo 绘制
df_tmp.to_csv('df_tmp.csv')
df_tmp